# Temple University Dataset

The Temple University Dataset is a large, open-access, dataset of clincal EEG data. 

Web link: https://www.isip.piconepress.com/projects/tuh_eeg/html/overview.shtml

Paper Overview: http://journal.frontiersin.org/article/10.3389/fnins.2016.00196/full

Ultimately, the goal will be to use this dataset, which is on a scale never really seen before in EEG datasets, to try and characterize and understand brain activity in a large cohort, across 'normal' brain activity, and disease. 

## TASKS:
The first set of tasks relates to working on a pre-processing pipeline for the data. Regardless of the set of analyses we ultimately choose, the data need to be organized, cleaned and pre-processed in such a way that we have confidence that the analyses we run reflect brain activity, not artifacts. A key goal of setting up this pipeline is that it be automatic and robust - given the number of subjects, we need to have confidence we can run this pipeline without manual supervision, and trust the data that comes out of it. 

Note: since we're using MNE, we don't want to end up with some information inside the MNE objects, and some outside them. As we develop these processes, let's try to keep a tight integration with MNE. 

### First Goal:
    - Fully clean a subject of the rest state, for example, run the PSD over the remaining channels, and trust it reflects brain activity.

### Preprocessing Tasks
Pre-processing pipeline:
  - Sort out the channel names and data types. 
      - Target output: a list of just the standard channel names (example: FP1, Cz, T4)
          - Also: Keep track of channel types. (Another list of ['EEG', 'EEG', 'EMG'])
              - Then: make sure we can put this 'back' into the MNE objects - update the channel info accordingly. 
  - What are the event markers? How do we line them up with the data? Can we epoch into different 'states'?
      - Target output: extracted epoch of resting data. 
  - How can we (automatically?) reject bad channels
      - We want to figure out MNE's available tools to reject channels, and start applying them.
      - Target output: For example subjects, channels we don't trust (visually) should be able to be 'dropped' (marked as bad so that they are ignored by future processing) by running an automatic process. 
  - How can we (automatically?) reject bad time segments
      - We want to figure out MNE's available tools to reject time segments, and start applying them.
      - Target output: For example subjects, time segments we don't trust (visually) should be dropped (again, marked bad to be ignored) by running an automatic process. 
  - How can we (automatically?) deal with stereotyped artifacts - such as eye blinks, heartbeats, saccades
      - Here I mean artifacts that we can correct for, different from time segments we decide we need to avoid.
      - We want to figure out MNE's available tools to correct for artifacts, and start applying them.
      - Target output: This depends slightly more on what is available (in the data, and in MNE), but ideally, for heartbeat and eye blinks (perhaps others), we would like to be able to automatically label them and 'subtract' them out from the data in some way - such that we can keep, and still use, these segments of data. 
      
Final outcome: A cleaned segment of resting data, ready for analysis, for 1 or 2 subjects. Working through this process will be very manually supervised, but hopefully by the time it's finished, the whole process can be re-run automatically, without human input, and this process can also be applied to other subjects. 
  
### Database Work (2nd level of task)
We will need utility functions to work with the database, to list subjects, find subjects, get paths, etc. 
- We'll work on this mainly when we want to scale up the number of subjects. Don't start it yet, I have some example code from other projects that can form a basis of this. 


### Metadata Work (3rd level of task)
Ultimately, we probably want some tools to be able to automatically scrape the clinical notes, and extract key parameters, like age, sex, and diagnoses. 
- This is not an immediate priority, as it will become more relevant as we fine-tune the analyses. We might, for example, start with the clearly labelled 'epilepsy vs non-epilepsy' group that is available, before we try to do large-scale mining of the clinical notes. 

## Notes:
- This first level of work is very pre-process-y, but required to get started with this data. Once we have a couple subjects set up with something that works at least reasonably well, we'll move on to more analysis stuff, and really get into the data, so that we're not stuck forever on pre-processing. 
- The goal is to get a rough version of the full pipeline, allowing us to 'see' the whole project, and find any major hurdles. It also, in practice, lets us try out each part of the project first, without getting bogged down at any particular step. 
- Eventually, we will have to double back and make sure each step is really robust.
- Also, soon, we will work a bit on setting up our codebase as a proper module, for organization and scalability. 

In [1]:
# Imports
%matplotlib qt
import mne
import os

In [15]:
# Set up paths

# This base path will need updating
base_path = 'C:\\Users\\Luyanda Mdanda\\Documents\\Research\\EEG_Dat'

# These should stay the same
#subj_dat_fname = '._3501_resampled.set'
subj_dat_fname = '3501_resampled.set'

In [16]:
# Read in an example subject of data (subject chosen randomly)
full_path = os.path.join(base_path, subj_dat_fname)
eeg_dat = mne.io.read_raw_eeglab(full_path)

#eeg_dat.set_eeg_reference()

Events like the following will be dropped entirely: ['MissTrial', 'Peak_Pres_catch', 'DECIMAL pressed', 'Peak_Pres_thresh', 'Sham_Pres_threshold'], 41 in total
2288/2998 event codes could not be mapped to integers. Use the 'event_id' parameter to map such events manually.
Reading C:\Users\Luyanda Mdanda\Documents\Research\EEG_Dat\3051_resampled.fdt


<ipython-input-16-be6a13d555ee>:3: RuntimeWarning: Events like the following will be dropped entirely: ['MissTrial', 'Peak_Pres_catch', 'DECIMAL pressed', 'Peak_Pres_thresh', 'Sham_Pres_threshold'], 41 in total
  eeg_dat = mne.io.read_raw_eeglab(full_path)
<ipython-input-16-be6a13d555ee>:3: RuntimeWarning: 2288/2998 event codes could not be mapped to integers. Use the 'event_id' parameter to map such events manually.
  eeg_dat = mne.io.read_raw_eeglab(full_path)


In [18]:
# verify that data has loaded correctly
eeg_dat.info

<Info | 16 non-empty fields
    bads : 'list | 0 items
    buffer_size_sec : 'float | 1.0
    ch_names : 'list | Fp1, Fp2, F7, F3, Fz, F4, F8, FC5, FC1, FC2
    chs : 'list | 65 items (EEG: 64, STIM: 1)
    comps : 'list | 0 items
    custom_ref_applied : 'bool | False
    dev_head_t : 'mne.transforms.Transform | 3 items
    events : 'list | 0 items
    highpass : 'float | 0.0 Hz
    hpi_meas : 'list | 0 items
    hpi_results : 'list | 0 items
    lowpass : 'float | 250.0 Hz
    meas_date : 'numpy.ndarray | 1969-12-31 16:00:00
    nchan : 'int | 65
    projs : 'list | 0 items
    sfreq : 'float | 500.0 Hz
    acq_pars : 'NoneType
    acq_stim : 'NoneType
    ctf_head_t : 'NoneType
    description : 'NoneType
    dev_ctf_t : 'NoneType
    dig : 'NoneType
    experimenter : 'NoneType
    file_id : 'NoneType
    filename : 'NoneType
    hpi_subsystem : 'NoneType
    kit_system_id : 'NoneType
    line_freq : 'NoneType
    meas_id : 'NoneType
    proj_id : 'NoneType
    proj_name : 'NoneTyp

In [17]:
# Check out the info object for this subject
eeg_dat.ch_names

['Fp1',
 'Fp2',
 'F7',
 'F3',
 'Fz',
 'F4',
 'F8',
 'FC5',
 'FC1',
 'FC2',
 'FC6',
 'T7',
 'C3',
 'Cz',
 'C4',
 'T8',
 'TP9',
 'CP5',
 'CP1',
 'CP2',
 'CP6',
 'TP10',
 'P7',
 'P3',
 'Pz',
 'P4',
 'P8',
 'PO9',
 'O1',
 'Oz',
 'O2',
 'PO10',
 'AF7',
 'AF3',
 'AF4',
 'AF8',
 'F5',
 'F1',
 'F2',
 'F6',
 'FT9',
 'FT7',
 'FC3',
 'FC4',
 'FT8',
 'FT10',
 'C5',
 'C1',
 'C2',
 'C6',
 'TP7',
 'CP3',
 'CPz',
 'CP4',
 'TP8',
 'P5',
 'P1',
 'P2',
 'P6',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'STI 014']

In [21]:
# Example of one way to start fixing up the channels
#  Priority again is to keep everything consistent with MNE, so first check for MNE functions that can do 
#   things like this, and that can update this information insode the data.info object. 

from collections import defaultdict
ch_types = defaultdict(str)

channel_names = []
channel_types =[]
# lists to store the array names

processed = None
# checks if eeg_dat was altered

for ch in eeg_dat.ch_names:    
    if 'EEG' in ch:
        ch_types[ch] = 'eeg'
        processed = True
        
    elif 'EMG' in ch:
        ch_types[ch] = 'emg'
        # Keeps track that this is an EMG channels
        
    elif 'STI' in ch:
        ch_types[ch] = 'stim'
        # Keeps track that this is an STI channel
    
    else:
        ch_types[ch] = 'misc'
        # Keeps track of MISC channels
        
eeg_dat.set_channel_types(ch_types)
        
for ch in eeg_dat.ch_names:    
    if 'EEG' in ch:
        ch = ch[4:7]
        ch =''.join(e for e in ch if e.isalnum())
        channel_names.append(ch)
        channel_types.append('eeg')
        ch_types[ch] = 'eeg'
        
    elif 'EMG' in ch:
        ch = ch[4:7]
        ch =''.join(e for e in ch if e.isalnum())
        channel_names.append(ch)
        channel_types.append('emg')
        ch_types[ch] = 'emg'
        # Keeps track that this is an EMG channels
        
    elif 'STI' in ch:
        channel_names.append(ch)
        channel_types.append('stim')
        ch_types[ch] = 'stim'
        # Keeps track that this is an STI channel
    
    else:
        channel_names.append(ch)
        channel_types.append('misc')
        ch_types[ch] = 'misc'
        # Keeps track of MISC channels
        
# The EEG channels use the standard naming strategy.
# By supplying the 'montage' parameter, approximate locations
montage = 'standard_1020'

if processed:
    info = mne.create_info(channel_names, eeg_dat.info['sfreq'], channel_types, montage)
    print(info)
    
else:
    print("No preprocessing on types done, data seems to be well formatted")

No preprocessing on types done.


In [ ]:
# Tests if information has been correctly organized
eeg_dat.info

In [ ]:
eeg_dat.plot()

In [ ]:
eeg_dat.load_data()

# Low pass filtering below 1 Hz
# High pass filtering above 40 Hz
eeg_dat.filter( 1, 40, filter_length='auto')
eeg_dat.plot_psd(area_mode='range', tmax=34.0)

In [ ]:
type(eeg_dat._data[0:])
eeg_dat._data[0, :].shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 

plt.plot(eeg_dat._data[0,0:500])
# plt.plot(eeg_dat._data[,0:500])

In [ ]:
eeg_dat.info

In [ ]:
import numpy as np
mins = []
maxs = []

nChans = eeg_dat.info['nchan']
nTimes = 34

dat = np.ones([nChans, nTimes])

for i in range(nChans):
   
    # Pull out a row (which is a channel)
    chan_data = eeg_dat._data[i, :]
   
    # Get the min and max of the row (channel)
    mins.append(min(chan_data))
    maxs.append(max(chan_data))

# Calculate ranges
ranges = []
for mi, ma in zip(mins, maxs):
    ranges.append(ma - mi)
    
# Calculate average range to set new threshold
avrange = sum(ranges)/len(ranges)

print (avrange)

In [ ]:
# Creates events at a 2 second as current data doesn't have any avaialble
events = mne.make_fixed_length_events(eeg_dat, 0, start = 0, stop = 34 , duration = 2, first_samp = True)

event_id = 0
tmin = 0 # start of each epoch 
tmax = 2 # end of each epoch (2000ms after the trigger)
baseline = None  # means from the first instant to t = 0
reject = dict( eeg = 0.000121364317584) #used average range of channels

picks_eeg = mne.pick_types(eeg_dat.info, meg=False, eeg=True, eog=True,
                           stim=False, exclude='bads')

epochs = mne.Epochs(eeg_dat, events, event_id, tmin, tmax, proj=True,
                    picks=picks_eeg, baseline=baseline, reject = reject,
                    reject_by_annotation=True, add_eeg_ref=False) ;

# Observation Notes for Tom
I think this clearly indicates that EKG1, FP1, FP2 are bad channels. So I think I can go ahead and manually label these as bad???


In [ ]:
eeg_dat.info['bads'] = ['EEG FP1-REF','EEG FP2-REF', 'EEG EKG1-REF']

In [ ]:
eeg_dat.plot()

In [ ]:
# Creates events at a 2 second as current data doesn't have any avaialble
events = mne.make_fixed_length_events(eeg_dat, 0, start = 0, stop = 34 , duration = 2, first_samp = True)

event_id = 0
tmin = 0 # start of each epoch 
tmax = 2 # end of each epoch (2000ms after the trigger)
baseline = None  # means from the first instant to t = 0
reject = dict( eeg = 0.000121364317584) #used average range of channels

picks_eeg = mne.pick_types(eeg_dat.info, meg=False, eeg=True, eog=True,
                           stim=False, exclude='bads')

epochs = mne.Epochs(eeg_dat, events, event_id, tmin, tmax, proj=True,
                    picks=picks_eeg, baseline=baseline, reject = reject,
                    reject_by_annotation=True, add_eeg_ref=False) ;

In [ ]:
epochs.drop_bad()

1.) Run Psd over each good channel
2.) Find matrix of psd per channel
3.) Parse those directl into FOOF


    Calc psd across trials then ave across that
....4.) Automate to set up some sort of comparrision



In [ ]:
epochs.plot_psd

In [ ]:
epochs.info

# Check: New Epochs has no bad channels included.

In [ ]:
epochs._data??

Type:        NoneType
String form: None
Docstring:   <no docstring>

In [ ]:
type(epochs)

In [ ]:
epochs._get_data()
# Load all data, dropping bad epochs along the way

# Promising.... if I had better understanding of what I was looking at

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
plt.plot(epochs._get_data())


# plt.plot(eeg_dat._data[,0:500])

# Get PSDs per channel

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Axes3D.scatter(xs, ys, zs=0, zdir='z', s=20, c=None, depthshade=True, *args, **kwargs)